In [1]:
#!/usr/bin/env python3
import numpy as np
import open3d as o3d
from pyorbbecsdk import Pipeline, Config, OBSensorType

# 1) Start camera + enable depth only
pipeline = Pipeline()
cfg      = Config()
dp_list  = pipeline.get_stream_profile_list(OBSensorType.DEPTH_SENSOR)
dp       = dp_list.get_default_video_stream_profile()
cfg.enable_stream(dp)
pipeline.start(cfg)

# 2) Grab one frame (timeout=1 s)
frames = pipeline.wait_for_frames(1000)
if frames is None:
    raise RuntimeError("No depth frame")

d = frames.get_depth_frame()
h, w = d.get_height(), d.get_width()

# 3) Convert to NumPy (uint16 mm) and wrap in O3D Image
depth_np = np.frombuffer(d.get_data().tobytes(), dtype=np.uint16).reshape((h, w))
depth_o3d = o3d.geometry.Image(depth_np)

# 4) Build O3D intrinsic from your profile
intr = dp.get_intrinsic()
o3d_intr = o3d.camera.PinholeCameraIntrinsic(
    width=w, height=h,
    fx=intr.fx, fy=intr.fy,
    cx=intr.cx, cy=intr.cy
)

# 5) Unproject into a PointCloud
pcd = o3d.geometry.PointCloud.create_from_depth_image(
    depth_o3d,
    o3d_intr,
    depth_scale=1000.0,   # because data is in millimeters
    depth_trunc=3.0,      # drop anything beyond 3 m
    stride=1
)

print(f"Unprojected {len(pcd.points)} points")

# 6) (Optional) visualize in a standalone window
o3d.visualization.draw_geometries([pcd])

pipeline.stop()


load extensions from /home/bradley/anaconda3/envs/o3d/lib/python3.10/site-packages/extensions
Unprojected 0 points
[Open3D WARNING] GLFW Error: Wayland: The platform does not support setting the window position
[Open3D WARNING] Failed to initialize GLEW.
[Open3D WARNING] [DrawGeometries] Failed creating OpenGL window.


In [3]:
from pyorbbecsdk import Pipeline, Config, OBSensorType
import numpy as np

depth_profiles = Pipeline().get_stream_profile_list(OBSensorType.DEPTH_SENSOR)
results = []

for idx, prof in enumerate(depth_profiles):
    print(f"\nTesting profile [{idx}]: {prof.get_format().name}, "
          f"{prof.get_width()}×{prof.get_height()} @ {prof.get_fps()}fps")
    # start a fresh pipeline for each profile
    pipe = Pipeline()
    cfg  = Config()
    cfg.enable_stream(prof)
    pipe.start(cfg)
    frames = pipe.wait_for_frames(500)
    if frames:
        d = frames.get_depth_frame()
        if d:
            h, w   = d.get_height(), d.get_width()
            depth  = np.frombuffer(d.get_data().tobytes(), dtype=np.uint16).reshape((h, w))
            non0   = np.count_nonzero(depth)
            print(f"  → non‑zero pixels: {non0} / {h*w}, min={depth.min()}, max={depth.max()}")
            results.append((idx, non0, depth.min(), depth.max()))
        else:
            print("  → no depth_frame")
    else:
        print("  → no frames")
    pipe.stop()

print("\nSummary of non‑zero results:")
for idx, non0, lo, hi in results:
    prof = depth_profiles[idx]
    print(f"• [{idx}] {prof.get_width()}×{prof.get_height()} @{prof.get_fps()}fps: "
          f"{non0} pixels, depth range {lo}–{hi}")




Testing profile [0]: Y16, 640×576 @ 30fps
  → no frames

Testing profile [1]: Y16, 1024×1024 @ 15fps
  → no frames

Testing profile [2]: Y16, 1024×1024 @ 5fps
  → no frames

Testing profile [3]: Y16, 640×576 @ 25fps
  → no frames

Testing profile [4]: Y16, 640×576 @ 15fps
  → no frames

Testing profile [5]: Y16, 640×576 @ 5fps
  → no frames

Testing profile [6]: Y16, 512×512 @ 30fps
  → no frames

Testing profile [7]: Y16, 512×512 @ 25fps
  → no frames

Testing profile [8]: Y16, 512×512 @ 15fps
  → no frames

Testing profile [9]: Y16, 512×512 @ 5fps
  → no frames

Testing profile [10]: Y16, 320×288 @ 30fps
  → no frames

Testing profile [11]: Y16, 320×288 @ 25fps
  → non‑zero pixels: 0 / 92160, min=0, max=0

Testing profile [12]: Y16, 320×288 @ 15fps
  → no frames

Testing profile [13]: Y16, 320×288 @ 5fps
  → no frames


RuntimeError: ob_stream_profile_list_get_profile: index out of range!